# python接口包装

我们一般用C的一个主要目的就是为python提供扩展,在python攻略中已经介绍了使用cython为python写扩展的方式,但这种方式一般是以python为主体的,而如果以C为主体,那么通常就和这种的方式有所区别,具体在:使用cython的包装模式,通过`pyd`文件声明接口,再使用`pyx`模块将项目包装为面向对象的python模块.这种方式

## 使用cython包装模式

为什么使用这种方式呢?主要是目前来看cython的包装模式是几个主流python实现都支持的方式,而且相对简单.接着我们还以binary vector为例进行包装

In [5]:
%%writefile ../codes/C1_C_tools/cython_wrapper/source/include/binary_vector.h
/* $Id$ */
#ifndef BINARY_VECTOR_H
#define BINARY_VECTOR_H//一般是文件名的大写 头文件结尾写上一行

struct BINARY_VECTOR {
    float x;
    float y;
};

typedef struct BINARY_VECTOR *BINARY_VECTOR_P;

BINARY_VECTOR_P VEC_new(void);
BINARY_VECTOR_P VEC_init(float x,float y);
void VEC_del(BINARY_VECTOR_P);

float VEC_mod(BINARY_VECTOR_P);
BINARY_VECTOR_P VEC_add(BINARY_VECTOR_P,BINARY_VECTOR_P);
float VEC_mul(BINARY_VECTOR_P ,BINARY_VECTOR_P);


#endif

Overwriting ../codes/C1_C_tools/cython_wrapper/source/include/binary_vector.h


In [6]:
%%writefile ../codes/C1_C_tools/cython_wrapper/source/src/binary_vector.c
#include <stdlib.h>
#include <math.h>
#include "binary_vector.h"

#define T BINARY_VECTOR_P


T VEC_new(void){
    T ptr;
    ptr = (T) malloc(((long)sizeof *(ptr)));
    ptr->x = 0.0;
    ptr->y = 0.0;
    return ptr;
}

T VEC_init(float x,float y){
    T ptr;
    ptr = (T) malloc(((long)sizeof *(ptr)));
    ptr->x = x;
    ptr->y = y;
    return ptr;
}

void VEC_del(T ptr){
    if (ptr) free(ptr);
}

T VEC_add(T a, T b) {
    float x = a->x + b->x;
    float y = a->y + b->y;
    T result;
    result = VEC_new();
    result->x = x;
    result->y = y;
    return result;
}

float VEC_mul(T a, T b) {
    float result = a->x * b->x + a->y * b->y;
    return result;
}

float VEC_mod(T a){
    float result = sqrt(a->x*a->x+a->y*a->y);
    return result;
}
#undef T

Overwriting ../codes/C1_C_tools/cython_wrapper/source/src/binary_vector.c


In [24]:
%%writefile ../codes/C1_C_tools/cython_wrapper/python/binaryvector.pyx
#cython: language_level=3
# distutils: language = c
# distutils: sources = binary_vector.c


cdef extern from "binary_vector.h":
    struct BINARY_VECTOR:
        float x
        float y
        
    ctypedef BINARY_VECTOR *BINARY_VECTOR_P
    
    BINARY_VECTOR_P VEC_init(float,float)
    void VEC_del(BINARY_VECTOR_P)
    
    float VEC_mod(BINARY_VECTOR_P)
    BINARY_VECTOR_P VEC_add(BINARY_VECTOR_P,BINARY_VECTOR_P)
    float VEC_mul(BINARY_VECTOR_P,BINARY_VECTOR_P)

cdef class BinaryVector:
    cdef BINARY_VECTOR_P _vec
    
    def __cinit__(self,float x,float y):
        self._vec = VEC_init(x,y)
        if self._vec is NULL:
            raise MemoryError
        
    def __dealloc__(self):
        if self._vec is not NULL:
            VEC_del(self._vec)
            
    property x:
        def __get__(self):
            return self._vec.x
        def __set__(self,new_x):
            self._vec.x = new_x
    property y:
        def __get__(self):
            return self._vec.y
        def __set__(self,new_y):
            self._vec.y = new_y
    cpdef mul(self,BinaryVector that):
        return VEC_mul(self._vec,that._vec)
            
    def __mul__(self,BinaryVector that):
        return self.mul(that)
        
    def add(self,BinaryVector that):
        cdef BINARY_VECTOR_P ptr
        ptr = VEC_add(self._vec,that._vec)
        result = BinaryVector(ptr.x,ptr.y)
        if ptr is not NULL:
            VEC_del(ptr)
        return result
    def __add__(self,BinaryVector that):
        return self.add(that)
    def mod(self):
        return VEC_mod(self._vec)

Overwriting ../codes/C1_C_tools/cython_wrapper/python/binaryvector.pyx


In [23]:
%%writefile ../codes/C1_C_tools/cython_wrapper/python/setup.py
from distutils.core import setup
from pathlib import Path
from distutils.extension import Extension
from Cython.Build import cythonize
from Cython.Distutils import build_ext

source_dir = Path("../source")
src_dir = source_dir.joinpath("src/binary_vector.c")
inc_dir = source_dir.joinpath("include")
extension = Extension(
    "binaryvector",
    sources=["binaryvector.pyx",str(src_dir)],
    include_dirs=[str(inc_dir)]
)

setup(
    cmdclass={'build_ext': build_ext},
    ext_modules=cythonize(extension),
)

Overwriting ../codes/C1_C_tools/cython_wrapper/python/setup.py
